## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T15', 'T29', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Save a copy of the application_df for later modification in attempt 3
attempt_3_app_df = application_df.copy()

# Convert original dataframe's categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimization Attempt 1

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               8800      
                                                                 
 dense_1 (Dense)             (None, 40)                8040      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 16881 (65.94 KB)
Trainable params: 16881 (65.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5686 - accuracy: 0.7238
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7301
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5490 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5478 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5472 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5641 - accuracy: 0.7313 - 727ms/epoch - 3ms/step
Loss: 0.5640778541564941, Accuracy: 0.7313119769096375


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_Attempt1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Optimization Attempt 2

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 100

nn_attempt2 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_attempt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_attempt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_attempt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_attempt2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 200)               8800      
                                                                 
 dense_4 (Dense)             (None, 40)                8040      
                                                                 
 dense_5 (Dense)             (None, 100)               4100      
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21041 (82.19 KB)
Trainable params: 21041 (82.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn_attempt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_attempt2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5676 - accuracy: 0.7232
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7310
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_attempt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5711 - accuracy: 0.7289 - 461ms/epoch - 2ms/step
Loss: 0.571144700050354, Accuracy: 0.728863000869751


In [21]:
# Export our model to HDF5 file
nn_attempt2.save("AlphabetSoupCharity_Optimization_Attempt2.h5")

## Optimization Attempt 3

### Extra Preprocessing

In [22]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
attempt_3_app_df = attempt_3_app_df.drop(columns=["STATUS"])

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
attempt_3_app_df = pd.get_dummies(attempt_3_app_df)

In [24]:
# Split our preprocessed data into our features and target arrays
y_attempt_3 = attempt_3_app_df['IS_SUCCESSFUL']
X_attempt_3 = attempt_3_app_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_attempt_3, y_attempt_3, random_state=1)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Third Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 40

nn_attempt3 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_attempt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_attempt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_attempt3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 200)               8600      
                                                                 
 dense_8 (Dense)             (None, 40)                8040      
                                                                 
 dense_9 (Dense)             (None, 1)                 41        
                                                                 
Total params: 16681 (65.16 KB)
Trainable params: 16681 (65.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn_attempt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn_attempt3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5694 - accuracy: 0.7213
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7329
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7336
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_attempt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5620 - accuracy: 0.7292 - 465ms/epoch - 2ms/step
Loss: 0.5619750618934631, Accuracy: 0.7292128205299377


In [30]:
# Export our model to HDF5 file
nn_attempt3.save("AlphabetSoupCharity_Optimization_Attempt3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
